# 03 - Modelo con preprocesado y SVMEste notebook carga los datos originales, aplica el preprocesamiento definido en el notebook 02 y entrena un modelo SVM para predecir el rendimiento global.

In [ ]:
import numpy as npimport pandas as pdimport seaborn as snsimport matplotlib.pyplot as pltfrom sklearn.model_selection import train_test_splitfrom sklearn.svm import SVCfrom sklearn.metrics import accuracy_score, classification_report, confusion_matriximport joblibsns.set(style="whitegrid")

In [ ]:
students_df = pd.read_csv("train.csv")print("Dimensiones del dataset original:", students_df.shape)students_df.head()

In [ ]:
faltantes = students_df.isnull().sum().sort_values(ascending=False)faltantes = faltantes[faltantes > 0]plt.figure(figsize=(8,4))sns.barplot(x=faltantes.values, y=faltantes.index)plt.title("Valores faltantes por columna")plt.xlabel("Cantidad")plt.show()faltantes

In [ ]:
id_estudiantes = students_df["ID"].copy()students_df = students_df.drop("ID", axis=1)students_df = students_df.fillna("Miss")students_df.head()

## Importar preprocesamiento

In [ ]:
from preprocesado import preprocesar_pipeline

In [ ]:
df_proc = preprocesar_pipeline(    students_df.copy(),    y_col="RENDIMIENTO_GLOBAL",    id_col_name=None,    generar_csv_individual=False,    train=True,    prefijo_csv="train")print("Dimensiones después del preprocesamiento:", df_proc.shape)df_proc.head()

In [ ]:
tipo_columnas = df_proc.dtypes.value_counts()plt.figure(figsize=(5,4))sns.barplot(x=tipo_columnas.values, y=tipo_columnas.index)plt.title("Cantidad de columnas por tipo de dato")plt.show()tipo_columnas

In [ ]:
y = df_proc["RENDIMIENTO_GLOBAL"]X = df_proc.drop(columns=["RENDIMIENTO_GLOBAL"])id_estudiantes = id_estudiantes.loc[X.index]X.head()

In [ ]:
Xtr, Xts, ytr, yts, id_tr, id_ts = train_test_split(    X, y, id_estudiantes,    test_size=0.20,    random_state=42,    stratify=y)print("Train:", Xtr.shape, " | Test:", Xts.shape)pd.DataFrame({    "Conjunto": ["Train", "Test"],    "Filas": [Xtr.shape[0], Xts.shape[0]],    "Columnas": [Xtr.shape[1], Xts.shape[1]],})

In [ ]:
modelo_svm = SVC(    kernel="rbf",    C=3,    gamma="scale",    class_weight="balanced",    probability=True)modelo_svm.fit(Xtr, ytr)print("Modelo SVM entrenado correctamente.")

In [ ]:
y_pred = modelo_svm.predict(Xts)acc = accuracy_score(yts, y_pred)print("Accuracy:", acc)print("\nReporte de clasificación:")print(classification_report(yts, y_pred))cm = confusion_matrix(yts, y_pred)plt.figure(figsize=(6,5))sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")plt.title("Matriz de confusión - SVM")plt.ylabel("Real")plt.xlabel("Predicción")plt.show()

In [ ]:
joblib.dump(modelo_svm, "modelo_svm.joblib")print("Guardado como modelo_svm.joblib")

In [ ]:
resultado = pd.DataFrame({    "ID": id_estudiantes,    "RENDIMIENTO_GLOBAL": modelo_svm.predict(X)})resultado.to_csv("predicciones_svm.csv", index=False)resultado.head()